In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, TrainingArguments,Trainer,BertModel
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from datasets import load_dataset, load_metric

In [ ]:
from transformers import BertConfig, BertModel

# Load the existing configuration file for BERT base uncased
config = BertConfig.from_pretrained('bert-base-uncased')

In [ ]:
from torch.nn.utils import clip_grad_norm_
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

# Define the configuration for the smaller model
config = BertConfig(
    hidden_size=256,  # Decrease the hidden size
    num_attention_heads=4,  # Reduce the number of attention heads
    num_hidden_layers=4,  # Reduce the number of hidden layers
    intermediate_size=1024,  # Adjust the intermediate size accordingly
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1
)

# Create the smaller BERT model
model = BertForSequenceClassification(config)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Replace with your tokenizer
dataset = load_dataset('glue', 'sst2')


train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

train_dataset = train_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

val_dataset = val_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)


train_encodings = tokenizer(train_dataset["sentence"], truncation=True, padding=True)
train_labels = train_dataset["label"]

train_dataset = TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_labels)
)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

weight_decay = 0.01  # Adjust the weight decay parameter
dropout_prob = 0.1  # Adjust the dropout probability

# Fine-tune the model
model.train()
train_losses = []
val_losses = []
accuracies = []

for epoch in range(50): 
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Apply weight decay regularization
        l2_norm = sum(p.pow(2).sum() for p in model.parameters())
        loss += weight_decay * l2_norm

        total_loss += loss.item()

        loss.backward()

        # Apply gradient clipping to prevent exploding gradients
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    accuracy = total_correct / total_samples
    accuracies.append(accuracy)
    
    print(f"Epoch {epoch+1} - Avg Train Loss: {avg_train_loss:.4f}")
    print(f"Epoch {epoch+1} - Accuracy: {accuracy:.2%}")

    eval_encodings = tokenizer(val_dataset["sentence"], truncation=True, padding=True)
    eval_labels = val_dataset["label"]

    eval_dataset = TensorDataset(
        torch.tensor(eval_encodings["input_ids"]),
        torch.tensor(eval_encodings["attention_mask"]),
        torch.tensor(eval_labels)
    )
    eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

    model.eval()
    total_eval_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Apply weight decay regularization
            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            loss += weight_decay * l2_norm

            total_eval_loss += loss.item()

            _, predicted_labels = torch.max(outputs.logits, dim=1)
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    val_losses.append(avg_eval_loss)
    accuracy = total_correct / total_samples
    
    print("Avg Eval Loss: {:.4f}".format(avg_eval_loss))
    print("Accuracy: {:.2%}".format(accuracy))
    print()


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Epoch 1 - Avg Train Loss: 35.2816
Epoch 1 - Accuracy: 55.75%
Avg Eval Loss: 22.7971
Accuracy: 50.92%

Epoch 2 - Avg Train Loss: 21.0368
Epoch 2 - Accuracy: 55.78%
Avg Eval Loss: 19.9301
Accuracy: 50.92%

Epoch 3 - Avg Train Loss: 19.0416
Epoch 3 - Accuracy: 55.78%
Avg Eval Loss: 18.1867
Accuracy: 50.92%

Epoch 4 - Avg Train Loss: 17.3386
Epoch 4 - Accuracy: 55.78%
Avg Eval Loss: 16.5273
Accuracy: 50.92%

Epoch 5 - Avg Train Loss: 15.7231
Epoch 5 - Accuracy: 55.78%
Avg Eval Loss: 14.9525
Accuracy: 50.92%

Epoch 6 - Avg Train Loss: 14.1946
Epoch 6 - Accuracy: 55.78%
Avg Eval Loss: 13.4688
Accuracy: 50.92%

Epoch 7 - Avg Train Loss: 12.7542
Epoch 7 - Accuracy: 55.78%
Avg Eval Loss: 12.0740
Accuracy: 50.92%

Epoch 8 - Avg Train Loss: 11.4008
Epoch 8 - Accuracy: 55.78%
Avg Eval Loss: 10.7598
Accuracy: 50.92%

Epoch 9 - Avg Train Loss: 10.1285
Epoch 9 - Accuracy: 55.78%
Avg Eval Loss: 9.5295
Accuracy: 50.92%

Epoch 10 - Avg Train Loss: 8.9389
Epoch 10 - Accuracy: 55.78%
Avg Eval Loss: 8.3834

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters: ", total_params)

Total Parameters:  11171074


In [ ]:
# Save the fine-tuned model
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json')

In [ ]:
from torch.nn.utils import clip_grad_norm_
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

# Define the configuration for the smaller model
config = BertConfig(
    hidden_size=256,  # Decrease the hidden size
    num_attention_heads=4,  # Reduce the number of attention heads
    num_hidden_layers=4,  # Reduce the number of hidden layers
    intermediate_size=1024,  # Adjust the intermediate size accordingly
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1
)

# Create the smaller BERT model
model = BertForSequenceClassification(config)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Replace with your tokenizer

dataset = load_dataset('glue', 'sst2')


train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

train_dataset = train_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

val_dataset = val_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

train_encodings = tokenizer(train_dataset["sentence"], truncation=True, padding=True)
train_labels = train_dataset["label"]

train_dataset = TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_labels)
)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

weight_decay = 0.01  # Adjust the weight decay parameter
dropout_prob = 0.1  # Adjust the dropout probability
l1_lambda = 0.01  # L1 regularization lambda value

# Fine-tune the model
model.train()
train_losses = []
val_losses = []
accuracies = []

for epoch in range(50): 
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Apply weight decay regularization
        l2_norm = sum(p.pow(2).sum() for p in model.parameters())
        loss += weight_decay * l2_norm

        # Apply L1 regularization
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += l1_lambda * l1_norm

        total_loss += loss.item()

        loss.backward()

        # Apply gradient clipping to prevent exploding gradients
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    accuracy = total_correct / total_samples
    accuracies.append(accuracy)
    
    print(f"Epoch {epoch+1} - Avg Train Loss: {avg_train_loss:.4f}")
    print(f"Epoch {epoch+1} - Accuracy: {accuracy:.2%}")

    eval_encodings = tokenizer(val_dataset["sentence"], truncation=True, padding=True)
    eval_labels = val_dataset["label"]

    eval_dataset = TensorDataset(
        torch.tensor(eval_encodings["input_ids"]),
        torch.tensor(eval_encodings["attention_mask"]),
        torch.tensor(eval_labels)
    )
    eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

    model.eval()
    total_eval_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Apply weight decay regularization
            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            loss += weight_decay * l2_norm

            total_eval_loss += loss.item()

            _, predicted_labels = torch.max(outputs.logits, dim=1)
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    val_losses.append(avg_eval_loss)
    accuracy = total_correct / total_samples
    
    print("Avg Eval Loss: {:.4f}".format(avg_eval_loss))
    print("Accuracy: {:.2%}".format(accuracy))
    print()


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1 - Avg Train Loss: 583.3233
Epoch 1 - Accuracy: 55.47%
Avg Eval Loss: 22.2071
Accuracy: 50.92%

Epoch 2 - Avg Train Loss: 47.4247
Epoch 2 - Accuracy: 55.78%
Avg Eval Loss: 19.9830
Accuracy: 50.92%

Epoch 3 - Avg Train Loss: 39.8311
Epoch 3 - Accuracy: 55.78%
Avg Eval Loss: 18.2370
Accuracy: 50.92%

Epoch 4 - Avg Train Loss: 37.1340
Epoch 4 - Accuracy: 55.78%
Avg Eval Loss: 16.5747
Accuracy: 50.92%

Epoch 5 - Avg Train Loss: 34.5300
Epoch 5 - Accuracy: 55.78%
Avg Eval Loss: 14.9962
Accuracy: 50.92%

Epoch 6 - Avg Train Loss: 32.0189
Epoch 6 - Accuracy: 55.78%
Avg Eval Loss: 13.5042
Accuracy: 50.92%

Epoch 7 - Avg Train Loss: 29.6039
Epoch 7 - Accuracy: 55.78%
Avg Eval Loss: 12.1044
Accuracy: 50.92%

Epoch 8 - Avg Train Loss: 27.2803
Epoch 8 - Accuracy: 55.78%
Avg Eval Loss: 10.7861
Accuracy: 50.92%

Epoch 9 - Avg Train Loss: 25.0357
Epoch 9 - Accuracy: 55.78%
Avg Eval Loss: 9.5488
Accuracy: 50.92%

Epoch 10 - Avg Train Loss: 22.8722
Epoch 10 - Accuracy: 55.78%
Avg Eval Loss: 8.39